### Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import time
import random
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer,classification_report, cross_val_score

In [21]:
csv_file_path = "adult.csv"

In [22]:
df = pd.read_csv(csv_file_path, na_values=['?'])
df.shape

(48842, 15)

In [23]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [25]:
df.isnull().sum()

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

In [26]:
df_processed = df.copy()

SUBSET_SIZE = 5000

df_processed = df_processed.sample(n=SUBSET_SIZE, random_state=42).reset_index(drop=True)

TARGET_COLUMN = 'income'
NEW_TARGET_NAME = 'Outcome'

y = df_processed[TARGET_COLUMN]
X = df_processed.drop(TARGET_COLUMN, axis=1)
COLUMNS_TO_DROP = ['fnlwgt']
if 'ID' in X.columns:
    COLUMNS_TO_DROP.append('ID')
if 'policy_id' in X.columns:
    COLUMNS_TO_DROP.append('policy_id')

X = X.drop(columns=COLUMNS_TO_DROP, errors='ignore')

y = y.rename(NEW_TARGET_NAME)

if y.dtype == 'object':
    le = LabelEncoder()
    y = pd.Series(le.fit_transform(y), name=NEW_TARGET_NAME, index=y.index)

numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

all_identified_features = numerical_features + categorical_features
if len(all_identified_features) != X.shape[1]:
    print("Warning: Not all columns were classified as numerical or categorical!")
    print("Unclassified columns:", [col for col in X.columns if col not in all_identified_features])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

transformers = []
if numerical_features:
    transformers.append(('num', numerical_pipeline, numerical_features))
if categorical_features:
    transformers.append(('cat', categorical_pipeline, categorical_features))

if not transformers:
     raise ValueError("No numerical or categorical features identified for preprocessing.")


preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough')

X_train shape: (4000, 13)
y_train shape: (4000,)
X_test shape: (1000, 13)
y_test shape: (1000,)


In [27]:
def evaluate_classifier(model, X, y):
    y_pred = model.predict(X)
    if y.dtype not in [np.number, np.int64, np.float64]:
         print(f"Warning: Target labels are not numerical (dtype is {y.dtype}). Cannot calculate ROC AUC.")
         auc = np.nan
    else:
        try:
            if hasattr(model, 'predict_proba'):
                 y_prob = model.predict_proba(X)[:, 1]
                 if len(np.unique(y)) == 2:
                     auc = roc_auc_score(y, y_prob)
                 else:
                     auc = np.nan

            else:
                 auc = np.nan

        except Exception as e:
            print(f"Warning: Could not calculate ROC AUC. Error: {e}")
            auc = np.nan

    metrics = {
        'accuracy': accuracy_score(y, y_pred),
        'precision': precision_score(y, y_pred),
        'recall': recall_score(y, y_pred),
        'f1_score': f1_score(y, y_pred),
        'roc_auc': auc
    }
    return metrics

### SVC

In [28]:
svm_baseline_estimator = SVC(random_state=42, probability=True)

baseline_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svc', svm_baseline_estimator)
])

print(baseline_pipeline.named_steps['svc'].get_params())

start_time = time.time()
baseline_pipeline.fit(X_train, y_train)
train_time_baseline = time.time() - start_time

print(f"\nBaseline model training time: {train_time_baseline:.4f} seconds")

baseline_metrics = evaluate_classifier(baseline_pipeline, X_test, y_test)

for metric, value in baseline_metrics.items():
    print(f"{metric}: {value:.4f}")

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False}

Baseline model training time: 6.3279 seconds
accuracy: 0.8700
precision: 0.7865
recall: 0.6292
f1_score: 0.6991
roc_auc: nan


C:\Users\HP\AppData\Local\Temp\ipykernel_12184\1545063328.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if y.dtype not in [np.number, np.int64, np.float64]:


In [29]:
print("\n--- Traditional Tuning: Grid Search for SVM Parameters ---")

svm_gs_estimator = SVC(random_state=42, probability=True)

gs_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svc', svm_gs_estimator)
])

param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__gamma': [0.01, 0.1, 'scale'],
    'svc__kernel': ['rbf']
}

print(param_grid)

cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=gs_pipeline,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

start_time = time.time()
grid_search.fit(X_train, y_train)
tuning_time_gs = time.time() - start_time

best_params_gs = grid_search.best_params_
best_svm_model_gs = grid_search.best_estimator_

print(best_params_gs)
print(f"Best cross-validation score ({grid_search.scorer_}): {grid_search.best_score_:.4f}")

tuned_metrics_gs = evaluate_classifier(best_svm_model_gs, X_test, y_test)

for metric, value in tuned_metrics_gs.items():
    print(f"{metric}: {value:.4f}")


--- Traditional Tuning: Grid Search for SVM Parameters ---
{'svc__C': [0.1, 1, 10], 'svc__gamma': [0.01, 0.1, 'scale'], 'svc__kernel': ['rbf']}
Fitting 3 folds for each of 9 candidates, totalling 27 fits
{'svc__C': 10, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
Best cross-validation score (make_scorer(f1_score, response_method='predict', average=binary)): 0.6572
accuracy: 0.8660
precision: 0.7650
recall: 0.6375
f1_score: 0.6955
roc_auc: nan


C:\Users\HP\AppData\Local\Temp\ipykernel_12184\1545063328.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if y.dtype not in [np.number, np.int64, np.float64]:


In [30]:
print("Metric         | Baseline SVM | Grid Search Tuned SVM")
print("---------------------------------------------------")
all_metrics = sorted(list(set(baseline_metrics.keys()) | set(tuned_metrics_gs.keys())))

for metric in all_metrics:
    baseline_val = baseline_metrics.get(metric, np.nan)
    tuned_gs_val = tuned_metrics_gs.get(metric, np.nan)

    baseline_str = f"{baseline_val:.4f}" if pd.notna(baseline_val) else "N/A     "
    tuned_gs_str = f"{tuned_gs_val:.4f}" if pd.notna(tuned_gs_val) else "N/A     "

    print(f"{metric:<14} | {baseline_str}     | {tuned_gs_str}")

print(f"\nTuning Time (seconds): Baseline: {train_time_baseline:.4f}, Grid Search: {tuning_time_gs:.4f}")

Metric         | Baseline SVM | Grid Search Tuned SVM
---------------------------------------------------
accuracy       | 0.8700     | 0.8660
f1_score       | 0.6991     | 0.6955
precision      | 0.7865     | 0.7650
recall         | 0.6292     | 0.6375
roc_auc        | N/A          | N/A     

Tuning Time (seconds): Baseline: 6.3279, Grid Search: 29.7339


### Nature Inspired Optimization

#### Whale Optimization Algorithm WOA

A nature inspired meta-heuristic optimization algorithm which mimics the hunting behaviour of humpback whales. The algorithm is inspired by the bubble-net hunting strategy.

In [ ]:
def objective_function(params, X, y):

    C, gamma = params
    
    svm_model = SVC(
        C=C,
        gamma=gamma,
        kernel='rbf',
        random_state=42,
        probability=True
    )
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('svc', svm_model)
    ])
    
    cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y, cv=cv_strategy, scoring='f1', n_jobs=-1, error_score='raise')
    
    # Negative F1score since WOA minimizes
    return -np.mean(scores)

def whale_optimization_algorithm(objective_func, lb, ub, dim, num_agents, max_iter, X, y, patience=10):

    # Init
    lb = np.array(lb)
    ub = np.array(ub)
    X_positions = np.random.uniform(lb, ub, (num_agents, dim))
    Fitness = np.array([objective_func(x, X, y) for x in X_positions])
    
    X_best = X_positions[np.argmin(Fitness)]
    F_best = np.min(Fitness)
    
    # for early stopping
    stagnation_count = 0 
    last_best_fitness = F_best
    
    for t in range(max_iter):
        a = 2 - t * (2 / max_iter)
        
        for i in range(num_agents):
            r1, r2 = np.random.rand(), np.random.rand()
            A = 2 * a * r1 - a
            C = 2 * r2
            p = np.random.rand()
            l = np.random.uniform(-1, 1)
            
            if p < 0.5:
                if abs(A) < 1:
                    # Shrinking circle
                    D = abs(C * X_best - X_positions[i])  # d away from best
                    X_positions[i] = X_best - A * D
                else:
                    # Search for prey
                    rand_index = np.random.randint(0, num_agents)
                    X_rand = X_positions[rand_index]
                    D = abs(C * X_rand - X_positions[i])  # d from rand agent
                    X_positions[i] = X_rand - A * D
            else:
                # Spiral
                distance_to_leader = abs(X_best - X_positions[i])
                X_positions[i] = distance_to_leader * np.exp(l) * np.cos(2 * np.pi * l) + X_best
            
            X_positions[i] = np.clip(X_positions[i], lb, ub)
        
        Fitness = np.array([objective_func(x, X, y) for x in X_positions])
        
        # update the best sol
        for i in range(num_agents):
            if Fitness[i] < F_best:
                X_best = X_positions[i]
                F_best = Fitness[i]
        
        # check for convergence
        if F_best < last_best_fitness:
            stagnation_count = 0
            last_best_fitness = F_best
        else:
            stagnation_count += 1
        
        print(f"Iteration {t+1}/{max_iter}, Best Fitness: {-F_best:.4f}")
        
        # Stop early if no improvement
        if stagnation_count >= patience:
            print(f"Convergence reached. No improvement for {patience} iterations.")
            break
    
    return X_best, -F_best

In [46]:
# lower, upper bounds for C and gamma
lb = [0.1, 0.01]  
ub = [10, 1]
dim = 2
num_agents = 10
max_iter = 20
patience = 5

results = []

for run in range(30):
    print(f"\n--- Run {run + 1}/30 ---")
    
    random_seed = run
    random.seed(random_seed)
    np.random.seed(random_seed)
    
    start_time = time.time()
    best_params_woa, best_fitness_woa = whale_optimization_algorithm(
        objective_func=objective_function,
        lb=lb,
        ub=ub,
        dim=dim,
        num_agents=num_agents,
        max_iter=max_iter,
        X=X_train,
        y=y_train,
        patience=patience
    )
    tuning_time_woa = time.time() - start_time
    
    # Train the final SVM model with the best parameters
    C_best, gamma_best = best_params_woa
    final_svm_model = Pipeline([
        ('preprocessor', preprocessor),
        ('svc', SVC(
            C=C_best,
            gamma=gamma_best,
            kernel='rbf',
            random_state=42,
            probability=True
        ))
    ])
    
    final_svm_model.fit(X_train, y_train)
    
    y_pred = final_svm_model.predict(X_test)
    test_metrics = classification_report(y_test, y_pred, output_dict=True)
    
    # Save results for run
    results.append({
        'Run': run + 1,
        'Best Parameters': best_params_woa,
        'Best Cross-Validation F1-Score': best_fitness_woa,
        'Tuning Time (s)': tuning_time_woa,
        'Test Accuracy': test_metrics['accuracy'],
        'Test F1-Score': test_metrics['weighted avg']['f1-score'],
        'Test Precision': test_metrics['weighted avg']['precision'],
        'Test Recall': test_metrics['weighted avg']['recall']
    })
    
    print(f"Run {run + 1}: Best Parameters: {best_params_woa}, CV F1-Score: {best_fitness_woa:.4f}, "
          f"Tuning Time: {tuning_time_woa:.2f}s")

results_df = pd.DataFrame(results)

results_df.to_csv('woa_results_30_runs.csv', index=False)

print("\n--- Summary of 30 Runs ---")
print(results_df.describe())


--- Run 1/30 ---
Iteration 1/20, Best Fitness: 0.6564
Iteration 2/20, Best Fitness: 0.6643
Iteration 3/20, Best Fitness: 0.6651
Iteration 4/20, Best Fitness: 0.6651
Iteration 5/20, Best Fitness: 0.6651
Iteration 6/20, Best Fitness: 0.6651
Iteration 7/20, Best Fitness: 0.6651
Iteration 8/20, Best Fitness: 0.6651
Convergence reached. No improvement for 5 iterations. Stopping early.
Run 1: Best Parameters: [8.10985052 0.37551177], CV F1-Score: 0.6651, Tuning Time: 504.73s

--- Run 2/30 ---
Iteration 1/20, Best Fitness: 0.6608
Iteration 2/20, Best Fitness: 0.6608
Iteration 3/20, Best Fitness: 0.6609
Iteration 4/20, Best Fitness: 0.6617
Iteration 5/20, Best Fitness: 0.6617
Iteration 6/20, Best Fitness: 0.6624
Iteration 7/20, Best Fitness: 0.6632
Iteration 8/20, Best Fitness: 0.6632
Iteration 9/20, Best Fitness: 0.6632
Iteration 10/20, Best Fitness: 0.6632
Iteration 11/20, Best Fitness: 0.6632
Iteration 12/20, Best Fitness: 0.6632
Convergence reached. No improvement for 5 iterations. Stoppi

#### Simulated Annealing

In [ ]:
# Initialization: pop, temp, cooling rate
# Cooling Schedule
def simulated_annealing_optimization(pop_size, init_temp, cooling_rate):
    population = []
    best_solution = None
    ind = [random.uniform(lb[0], ub[0]), random.uniform(lb[1], ub[1])] # or choose initial solution
    population.append(ind)

def generate_neighbor(ind, temp_k):
    current_sol = 0
    new_sol = 0
    diff_of_sols = objective_function(new_sol) - objective_function(current_sol)
    
    if diff_of_sols <= 0:
        current_sol = new_sol
     
    else:
        if random.random > np.exp(diff_of_sols/temp_k):
            current_sol = new_sol

    return best_solution

In [71]:
def objective_function(params, X, y):

    C, gamma = params
    
    svm_model = SVC(
        C=C,
        gamma=gamma,
        random_state=42,
        probability=True
    )
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('svc', svm_model)
    ])
    
    cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y, cv=cv_strategy, scoring='f1', n_jobs=-1, error_score='raise')
    
    # negative F1 score
    return -np.mean(scores)

def simulated_annealing(initial_solution, temperature_schedule, max_iterations_per_temp, stopping_temperature, X, y):

    current_solution = initial_solution
    current_energy = objective_function(current_solution, X, y)
    best_solution = current_solution.copy()
    best_energy = current_energy
    
    k = 0
    stagnation_count = 0
    last_best_energy = best_energy
    
    while True:

        T_k = temperature_schedule(k)
        
        print(f"Iteration k={k}, Temperature T_k={T_k:.6f}, Best Energy={-best_energy:.4f}")
        
        # Check stopping criterion
        if T_k < stopping_temperature:
            break
        
        m = 0
        
        while m < max_iterations_per_temp:
            omega_prime = generate_neighbor(current_solution)
            
            delta_energy = objective_function(omega_prime, X, y) - current_energy
            
            if delta_energy <= 0:
                current_solution = omega_prime
                current_energy = objective_function(current_solution, X, y)
                
                if current_energy < best_energy:
                    best_solution = current_solution.copy()
                    best_energy = current_energy
            else:
                # exploration part of the algo
                acceptance_probability = np.exp(-delta_energy / T_k)
                if random.random() < acceptance_probability:
                    current_solution = omega_prime
                    current_energy = objective_function(current_solution, X, y)
            
            m += 1
                
        # (early stopping)
        if best_energy < last_best_energy:
            stagnation_count = 0 
            last_best_energy = best_energy
        else:
            stagnation_count += 1  

        if stagnation_count >= patience:
            print(f"Early stopping: No improvement for {patience} iterations.")
            break
        
        k += 1
    
    return best_solution, -best_energy

# N(omega) function
def generate_neighbor(solution):

    C, gamma = solution
    
    C_new = max(0.1, min(10, C + np.random.normal(0, 0.5)))
    gamma_new = max(0.01, min(1, gamma + np.random.normal(0, 0.05)))
        
    return [C_new, gamma_new]

In [72]:
lb = [0.1, 0.01]
ub = [10, 1] 
initial_solution = [random.uniform(lb[0], ub[0]), random.uniform(lb[1], ub[1])]
temperature_schedule = lambda k: 100 / (k + 1)  # Cooling schedule: T_k = 100 / (k + 1)
max_iterations_per_temp = 20
stopping_temperature = 1e-3
patience = 5

start_time = time.time()
best_params_sa, best_fitness_sa = simulated_annealing(
    initial_solution=initial_solution,
    temperature_schedule=temperature_schedule,
    max_iterations_per_temp=max_iterations_per_temp,
    stopping_temperature=stopping_temperature,
    X=X_train,
    y=y_train
)
tuning_time_sa = time.time() - start_time

C_best, gamma_best = best_params_sa

print("\n--- Simulated Annealing Results ---")
print(f"Best Parameters: C={C_best:.4f}, gamma={gamma_best:.4f}")
print(f"Best Cross-Validation F1-Score: {best_fitness_sa:.4f}")
print(f"Tuning Time: {tuning_time_sa:.2f} seconds")

# final SVC with best params
final_svm_model = Pipeline([
    ('preprocessor', preprocessor),
    ('svc', SVC(
        C=C_best,
        gamma=gamma_best,
        random_state=42,
        probability=True
    ))
])

final_svm_model.fit(X_train, y_train)

y_pred = final_svm_model.predict(X_test)
test_metrics = classification_report(y_test, y_pred, output_dict=True)

print("\nTest Set Metrics:")
print(classification_report(y_test, y_pred))

results = {
    'Best Parameters': best_params_sa,
    'Best Cross-Validation F1-Score': best_fitness_sa,
    'Tuning Time (s)': tuning_time_sa,
    'Test Accuracy': test_metrics['accuracy'],
    'Test F1-Score': test_metrics['weighted avg']['f1-score'],
    'Test Precision': test_metrics['weighted avg']['precision'],
    'Test Recall': test_metrics['weighted avg']['recall']
}

results_df = pd.DataFrame([results])
results_df.to_csv('sa_results_single_run.csv', index=False)

print("\nResults saved to 'sa_results_single_run.csv'")
print(results_df.describe())

Iteration k=0, Temperature T_k=100.000000, Best Energy=0.4785
Iteration k=1, Temperature T_k=50.000000, Best Energy=0.5018
Iteration k=2, Temperature T_k=33.333333, Best Energy=0.5018
Iteration k=3, Temperature T_k=25.000000, Best Energy=0.5018
Iteration k=4, Temperature T_k=20.000000, Best Energy=0.5605
Iteration k=5, Temperature T_k=16.666667, Best Energy=0.5975
Iteration k=6, Temperature T_k=14.285714, Best Energy=0.5975
Iteration k=7, Temperature T_k=12.500000, Best Energy=0.5975
Iteration k=8, Temperature T_k=11.111111, Best Energy=0.5975
Iteration k=9, Temperature T_k=10.000000, Best Energy=0.5975
Iteration k=10, Temperature T_k=9.090909, Best Energy=0.6214
Iteration k=11, Temperature T_k=8.333333, Best Energy=0.6651
Iteration k=12, Temperature T_k=7.692308, Best Energy=0.6651
Iteration k=13, Temperature T_k=7.142857, Best Energy=0.6651
Iteration k=14, Temperature T_k=6.666667, Best Energy=0.6651
Iteration k=15, Temperature T_k=6.250000, Best Energy=0.6651
Early stopping: No impr